* author：YHX 
* date  ：2022/12/19
* theme ：脑机接口--运动想象

In [ ]:
## 引入python库
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import mne
from mne.io import concatenate_raws, read_raw_edf  ## 读取edf格式数据
from mne.datasets import eegbci

In [ ]:
## 设置工作路径
os.chdir('D:\BMI\data\mne_data')  ## mne_data文件夹下就是S001、S002等
print(os.getcwd())

### 0 数据读入与切片
* 研究抽取32个观察样本，真实+想象的左右拳头

In [ ]:
from get_data_label import get_data_label    ########## 具体代码在最后  ##########

In [ ]:
## 数据读入列表 -- all_index_subjects 所有主体索引(1-109)    all_index_task 所有任务标签(1-14)
filePath = os.getcwd()
temp_index_subjects = os.listdir(filePath)
all_index_subjects = temp_index_subjects[5:-2]  ## 109

In [ ]:
subjects = all_index_subjects[:32] #[2]
path = filePath

event_id=dict( T0=1,T1=2, T2=3)  ## 主要看是否取 TO 静息状态  # T0=1,
tmin = 0 ; tmax = 4

In [ ]:
### 真实 左右拳
runs1 = [3, 7, 11]  
replacelist1 = ["rest","true_fist_left","true_fist_right"]

In [ ]:
get1 = get_data_label(subjects,runs1,path) 
data1 , event_data1 = get1.data_pair(event_id,tmin=0, tmax=4)
event_label1 = get1.get_event_label(event_data1,replacelist1)
event_label1 = np.array(event_label1)

In [ ]:
print(data1.shape, 2*"\n", event_label1.shape)

In [ ]:
### 想象 左右拳
runs2 = [4, 8, 12]  
replacelist2 = ["rest","img_fist_left","img_fist_right"]

In [ ]:
get2 = get_data_label(subjects,runs2,path) 
data2 , event_data2 = get1.data_pair(event_id,tmin=0, tmax=4)
event_label2 = get1.get_event_label(event_data2,replacelist2)
event_label2 = np.array(event_label2)

In [ ]:
print(data2.shape, 2*"\n", event_label2.shape)

In [ ]:
## 合并数据
data = np.concatenate((data1,data2),axis=0)
label = np.concatenate((event_label1,event_label2),axis=0)
print(data.shape, 2*"\n", label.shape)

In [ ]:
## data+label 保存
os.chdir("C:/Users/dell.000/机器学习_研一/期末论文_运动想象分类")

In [ ]:
fd = open("data.pkl", "rb")
result = pickle.load(fd)

In [ ]:
fl = open("label.pkl", "rb")
rlabel = pickle.load(fl)

##########################################################################################################################################

## 一、特征提取+经典分类器组合模型

### 1 数据划分

In [ ]:
## 引入python库
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
fd = open("data.pkl", "rb")
data = pickle.load(fd)
fd.close()
fl = open("label.pkl", "rb")
label = pickle.load(fl)
fl.close()

In [ ]:
print(data.shape, 2*"\n", label.shape)

In [ ]:
replacelist = ["rest","true_fist_left","true_fist_right","img_fist_left","img_fist_right"]
def get_event_label(label,replacelist):
    label_hat=[]
    need_index = []
    for i in range(len(label)):
        if label[i] == replacelist[0] :
            pass
            #label_hat.append(0)
            #need_index.append(i)
        elif label[i] == replacelist[1]:
            #pass
            label_hat.append(0)
            need_index.append(i)
        elif label[i] == replacelist[2]:
            #pass
            label_hat.append(1)
            need_index.append(i)
        elif label[i] == replacelist[3]:
            pass
            #label_hat.append(0)
            #need_index.append(i)
        elif label[i] == replacelist[4]:
            pass
            #label_hat.append(1)
            #need_index.append(i)
        else:
            print("存在错误")
    return label_hat,need_index

In [ ]:
label_hat,need_index = get_event_label(label,replacelist)

In [ ]:
data_hat = data[need_index,:,:]

In [ ]:
print(data_hat.shape, 2*"\n" , len(label_hat))

In [ ]:
## 具体数据划分
from sklearn.model_selection import train_test_split  # 样本划分 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data_hat, label_hat, random_state=0, test_size=0.1)

In [ ]:
print(X_train.shape, len(y_train), X_test.shape, len(y_test))

### 2 特征提取

In [ ]:
## 2.1 C_mean
from Mymodel import myC_Mean   ########## 具体代码在最后  ##########
modelC_mean = myC_mean(data)
X_train_C_mean = modelC_mean.get_C_mean_data(X_train)
X_test_C_mean = modelC_mean.get_C_mean_data(X_test)

In [ ]:
print(X_train_C_mean.shape, len(y_train), X_test_C_mean.shape, len(y_test))

In [ ]:
## 2.2 PCA
from Mymodel import myPCA     ########## 具体代码在最后  ##########
modelPCA = myPCA(X_train_C_mean)
n_components = modelPCA.get_PCA_nums()
modelPCA.get_plot_PVE()
modelPCA.get_plot_Cumulative_PVE()
X_train_pca,X_test_pca = modelPCA.fit(X_train_C_mean,X_test_C_mean, n_components=n_components)

In [ ]:
print(X_train_pca.shape, len(y_train), X_test_pca.shape, len(y_test))

### 3 经典分类模型搭建

In [ ]:
## 编写 获取混淆矩阵以及错误率函数
def get_confusion_matrix_result(y_test,y_predict):
    matrix_test = confusion_matrix(y_test,y_predict)
    print('混淆矩阵是：',"\n",matrix_test)
    
    print("估计的误判概率：\n", 
     "P(II|I)=",matrix_test[0][1]/sum(matrix_test[0]),"\n",
     "P(I|II)=",matrix_test[1][0]/sum(matrix_test[1]),)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report ### 混淆矩阵

In [ ]:
## LDA 费希尔判别
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis ### 费希尔判别

In [ ]:
lda = LinearDiscriminantAnalysis()
## 3.1 lda + C_mean
lda.fit(X_train_C_mean,y_train)
lda_C_mean_train_score = lda.score(X_train_C_mean,y_train)
print(lda_C_mean_train_score)

In [ ]:
lda_C_mean_test_score = lda.score(X_test_C_mean,y_test)
print(lda_C_mean_test_score)

In [ ]:
y_predict = lda.predict(X_test_C_mean)

In [ ]:
get_confusion_matrix_result(y_test,y_predict)

In [ ]:
################################################################################################################
lda = LinearDiscriminantAnalysis()
## 3.2 lda + PCA
lda.fit(X_train_pca,y_train)
lda_pca_train_score = lda.score(X_train_pca,y_train)
print(lda_pca_train_score)

In [ ]:
lda_pca_test_score = lda.score(X_test_pca,y_test)
print(lda_pca_test_score)

In [ ]:
y_predict = lda.predict(X_test_pca)

In [ ]:
get_confusion_matrix_result(y_test,y_predict)

In [ ]:
################################################################################################################
## SVM
from sklearn.svm import SVC

In [ ]:
svm = SVC()
## 3.1 svm + C_mean
svm.fit(X_train_C_mean,y_train)
svm_C_mean_train_score = svm.score(X_train_C_mean,y_train)
print(svm_C_mean_train_score)

In [ ]:
svm_C_mean_test_score = svm.score(X_test_C_mean,y_test)
print(svm_C_mean_test_score)

In [ ]:
y_predict = svm.predict(X_test_C_mean)

In [ ]:
get_confusion_matrix_result(y_test,y_predict)

In [ ]:
################################################################################################################
svm = SVC()
## 3.2 svm + PCA
svm.fit(X_train_pca,y_train)
svm_pca_train_score = svm.score(X_train_pca,y_train)
print(svm_pca_train_score)

In [ ]:
svm_pca_test_score = svm.score(X_test_pca,y_test)
print(svm_pca_test_score)

In [ ]:
y_predict = svm.predict(X_test_pca)

In [ ]:
get_confusion_matrix_result(y_test,y_predict)

### 4 学习曲线诊断

In [ ]:
from sklearn.model_selection import learning_curve ## 学习曲线
def get_learning_curve(function, X_train, y_train):
    train_sizes = np.linspace(0.1,1,num=40,endpoint=True) 
    train_sizes, train_scores, valida_scores = learning_curve(function, X_train, y_train, train_sizes= train_sizes,cv=5)
    print(train_scores.shape)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores,axis=1)
    validation_scores_mean = np.mean(valida_scores, axis=1)
    validation_scores_std = np.std(valida_scores, axis=1)
    plt.grid() ## 调整网格线
    plt.title (r"Learning Curves ( LinearDiscriminantAnalysis )")
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,   ## 曲线填充
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    plt.fill_between(train_sizes, validation_scores_mean - validation_scores_std,
                         validation_scores_mean + validation_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, '--', color="r", label="Training score")
    plt.plot(train_sizes, validation_scores_mean, '--', color="g", label="Cross-validation score")
    plt.legend(loc="best")  ## 图示位置“best”
    plt.show()

In [ ]:
train_val_X = np.concatenate((X_train_pca, X_test_pca), axis=0)  ## 合并数据
train_val_y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
get_learning_curve(lda,train_val_X,train_val_y)  ## LDA or SVM

## 二 多层一维卷积神经网络模型

In [ ]:
## 引入python库
import os
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import pickle
import torch
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## 生成label的one_hot
label_hat = np.array(label_hat).reshape((-1,1))
from sklearn.preprocessing import OneHotEncoder#导入One-Hot编码
## 使用One-Hot编码
enc = OneHotEncoder()
a=enc.fit_transform(label_hat)
label_list = enc.get_feature_names_out()
label_onehot =a.toarray()
label_list ## x0_1 左, 'x0_2' 右

### 1 数据集划分

In [ ]:
from sklearn.model_selection import train_test_split  # 样本划分 
X_train,X_val,y_train,y_val = train_test_split(data_hat, label_onehot, random_state=0, test_size=0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val, y_val, random_state=0, test_size=0.5)
print(X_train.shape, y_train.shape ,2*"\n", X_val.shape,y_val.shape,2*"\n",X_test.shape,y_test.shape)

In [ ]:
## 编写数据加载器
def myDataLoader_index(data,batch_size, shuffle):
    index_list = np.arange(data.shape[0])
    loader_index = DataLoader(index_list, batch_size=batch_size, shuffle=shuffle)
    return loader_index

In [ ]:
batch_size = 32

In [ ]:
train_loader_index = myDataLoader_index(X_train,batch_size=batch_size, shuffle=True)
val_loader_index = myDataLoader_index(X_val,batch_size=batch_size, shuffle=True)

### 2 模型搭建

In [ ]:
class myCNN1d(nn.Module):
    def __init__(self):
        super(myCNN1d, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 维度 [32，64, 641]
        self.cnn = nn.Sequential(
            nn.Conv1d(64, 32, 7, 1, 3),  # [32, 640]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 320]

            nn.Conv1d(32, 32, 7, 1, 3),  # [32, 320]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 160]

            nn.Conv1d(32, 32, 7, 1, 3),  # [32, 160]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 80]

        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 80, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

### 3 模型训练

In [ ]:
model = myCNN1d().to(device)
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # optimizer 使用 Adam
num_epoch = 30  # 迭代30次

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    train_count = 0.0
    val_acc = 0.0
    val_loss = 0.0
    val_count = 0.0

    model.train()  
    for i, data in enumerate(train_loader_index):
        Xtrian = torch.from_numpy(X_train[data,:,:]).to(torch.float32).to(device)
        ytrian = torch.from_numpy(y_train[data]).to(torch.float32).to(device)
        
        optimizer.zero_grad()  
        train_pred = model(Xtrian)  
        batch_loss = loss(train_pred, ytrian)  
        batch_loss.backward()  
        optimizer.step() 
        
        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == np.argmax(y_train[data], axis=1))  
        train_loss += batch_loss.item()
        train_count += Xtrian.shape[0]

    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader_index):
            Xval = torch.from_numpy(X_val[data,:,:]).to(torch.float32).to(device)
            yval = torch.from_numpy(y_val[data]).to(torch.float32).to(device)
            
            val_pred = model(Xval)
            batch_loss = loss(val_pred, yval)

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == np.argmax(y_val[data], axis=1))   
            val_loss += batch_loss.item()
            val_count += Xval.shape[0]
            
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
              (epoch + 1, num_epoch, time.time() - epoch_start_time, \
               train_acc / train_count, train_loss / train_count , val_acc / val_count, val_loss / val_count))
        train_acc_list.append(train_acc / train_count) 
        train_loss_list.append(train_loss / train_count) 
        val_acc_list.append(val_acc / val_count)
        val_loss_list.append(val_loss / val_count)

In [ ]:
## 画图Train Loss and Validation Loss
plt.figure(figsize=(8,6))  # 训练集的损失和验证集的损失，判断收敛和过拟合 
k=0
plt.plot(train_loss_list[k:],'ro-',label='Train Loss')   # 画3个epoch 之后的
#plt.plot(val_loss_list[k:],'bs-',label='Val Loss')
plt.title('Train Loss and Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
## 画图Train Acc and Validation Acc
plt.figure(figsize=(8,6))  # 训练集的损失和验证集的损失，判断收敛和过拟合 
k=0
plt.plot(train_acc_list[k:],'ro-',label='Train Acc')   # 画3个epoch 之后的
plt.plot(val_acc_list[k:],'bs-',label='Val Acc')
plt.title('Train Acc and Validation Acc')
plt.xlabel('epoch')
plt.ylabel('Acc')
plt.legend()

### 4 测试集分类

In [ ]:
## 合并训练集和验证集 重新训练
train_val_X = np.concatenate((X_train, X_val), axis=0)
train_val_y = np.concatenate((y_train, y_val), axis=0)
train_val_loader_index = myDataLoader_index(train_val_X,batch_size = batch_size, shuffle=True)

In [ ]:
model_best =  myCNN1d().to(device)
loss = nn.CrossEntropyLoss()  # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.0001)  # optimizer 使用 Adam
num_epoch = 30

loss_list = []
acc_list = []

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    train_count = 0.0
    
    model_best.train()
    for i, data in enumerate(train_val_loader_index):
        Xtrian = torch.from_numpy(train_val_X[data,:,:]).to(torch.float32).to(device)
        ytrian = torch.from_numpy(train_val_y[data]).to(torch.float32).to(device)
        
        optimizer.zero_grad()
        train_pred = model_best(Xtrian)
        batch_loss = loss(train_pred, ytrian)
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == np.argmax(train_val_y[data], axis=1))
        train_loss += batch_loss.item()
        train_count += Xtrian.shape[0]

        # 將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time() - epoch_start_time, \
           train_acc / train_count, train_loss / train_count))
    
    acc_list.append(train_acc / train_count) 
    loss_list.append(train_loss / train_count) 

In [ ]:
plt.figure(figsize=(12,5))  # 训练集的损失和验证集的损失，判断收敛和过拟合 
plt.subplot(1, 2, 1)     
k=0
plt.plot(loss_list[k:],'ro-',label='Loss')   # 画3个epoch 之后的
#plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.savefig('')
#plt.legend()

plt.subplot(1, 2, 2)     
k=0
plt.plot(acc_list[k:],'bs-',label='Acc')  # 画3个epoch 之后的
#plt.title('Acc')
plt.xlabel('epoch')
plt.ylabel('Acc')

In [ ]:
## 生成测试集结果
test_loader_index = myDataLoader_index(X_test,batch_size = batch_size, shuffle=False)
model_best.eval()
y_prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader_index):
        Xtest = torch.from_numpy(X_test[data,:,:]).to(torch.float32).to(device)
        
        test_pred = model_best(Xtest)
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            y_prediction.append(y)

In [ ]:
y_label =  np.argmax(y_test, axis=1)

In [ ]:
get_confusion_matrix_result(y_label,y_prediction)

### 附：封装代码函数

#### 1 get_data_label封装函数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.io import concatenate_raws, read_raw_edf  ## 读取edf格式数据
from mne.datasets import eegbci

class get_data_label():
    def __init__(self, subjects, runs, path):
        self.subjects = subjects
        self.runs = runs
        self.path = path
        
    def get_fnames(self):  ## subject 样本主体list， runs 任务list ， path 数据路径
        raw_fnames = list()
        for i in self.subjects:
            index_subjects = str(i).rjust(3,'0')  ## 如 1 —> 001   11 —> 011  101 —> 101
            # print(index_subjects)
            for ii in self.runs:
                index_task = str(ii).rjust(2,'0') 
                # print(index_task)
                temp_fname = str(self.path) + "\\" + str(index_subjects) + "\\" + str(index_subjects) +"R"+ str(index_task)+".edf"
                #print(temp_fname)
                raw_fnames.append(temp_fname)
            print("subject"+ str(i)+" 路径读取完毕")
        return(raw_fnames)
    
    def data_pair(self,event_id, tmin=0, tmax=4):  
        # raw：合并后数据
        # event_id：标签选择，tmin 标签点前时间 ，标签点后时间 
        # tmin ：标签点前时间，一般为-0.1/-0.2s，*160=16/32   —— 4.1*160 = 656
        # tmax ：标签点后时间，本数据库，取4s，4*160=640      —— 4.2*160 = 672    
        # num_files：输入edf文件数，为了解决因tmin<0而存在第一个静息状态无法获取问题(该问题会导致epoch量不等于label量)
        raw_fnames = self.get_fnames()
        num_files = len(raw_fnames)
        raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
        events, _ = mne.events_from_annotations(raw, event_id = event_id)  #, event_id = event_id
        print(events.shape)
        # 根据事件生成对应的Epochs数据
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True) #picks=picks
        # 获取epochs的所有数据
        epochs_data = epochs.get_data()    ##  epochs_data [subjects, channels , sample_node]

        ##event_label = epochs.events[:,-1]  
        if  "T0" in event_id:            ## 如果事件任务中包括“T0 = 1”即静止状态，可能存在系列情况
            print("Considered TO")
            # 获取event的所有数据
            if tmin >= 0 :
                event_data = events[:,2]
            else:                             ## 若tmim = -0.1,将会把第一个epoch作为错误epoch给去除
                num_files = num_files
                index_0 = [i*30 for i in range(num_files)]
                event_data = np.delete(events,index_0,axis=0)[:,2]
        else:
            print("Not considered TO")
            event_data = events[:,2]
        ## 是否对齐
        if epochs_data.shape[0] == event_data.shape[0]:
            print("Data and label pairing")
        else:
            print("Error:data and label not pairing")
        return epochs_data,event_data
    
    def get_event_label(self,event_data,replacelist):
        event_label=[]
        event_data = event_data-1
        for i in range(len(event_data)):
            if event_data[i] == 0 :
                event_label.append(replacelist[0])
            elif event_data[i] == 1 :
                event_label.append(replacelist[1])
            elif event_data[i] == 2 :
                event_label.append(replacelist[2])
            else:
                print("存在错误")
        return(event_label)

#### 2 Mymodel封装函数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mne.decoding import CSP

class myC_mean():
    def __init__(self,data):
        self.data = data
        
    def get_C_mean_data(self):
        data_mean = self.data.mean(axis=1,keepdims=True) 
        data_C_mean = data_mean.squeeze(1)
        return(data_C_mean)

class myPCA():
    def __init__(self,data):
        self.data = data
        self.model_PCA = PCA()
        self.model_PCA.fit(self.data)
        
    def get_PCA_nums(self):
        explained_variance_ratio =self.model_PCA.explained_variance_ratio_.cumsum()
        index = np.argmax((explained_variance_ratio >0.9)) 
        print("方差解释能力达到90%以上至少需要：",index+1,"个主成分")
        return(index+1)
    def get_plot_PVE(self):
        #可视化  画累计百分比，这样可以判断选几个主成分
        plt.plot(self.model_PCA.explained_variance_ratio_, 'o-')
        plt.xlabel('Principal Component')
        plt.ylabel('Proportion of Variance Explained')
        plt.title('PVE')
    def get_plot_Cumulative_PVE(self):   
        # 解释到90%以上了
        plt.plot(self.model_PCA.explained_variance_ratio_.cumsum(), 'o-')
        plt.xlabel('Principal Component')
        plt.ylabel('Cumulative Proportion of Variance Explained')
        plt.axhline(0.9, color='k', linestyle='--', linewidth=1)
        plt.title('Cumulative PVE')
        
    def fit(self,X_train,X_test,n_components):
        model = PCA(n_components = n_components)
        model.fit(X_train)
        #得到主成分得分
        X_train_pca = model.transform(X_train)
        X_test_pca = model.transform(X_test)
        return X_train_pca,X_test_pca

class myCNN1d(nn.Module):
    def __init__(self):
        super(myCNN1d, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 维度 [32，64, 641]
        self.cnn = nn.Sequential(
            nn.Conv1d(64, 32, 7, 1, 3),  # [32, 640]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 320]

            nn.Conv1d(32, 32, 7, 1, 3),  # [32, 320]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 160]

            nn.Conv1d(32, 32, 7, 1, 3),  # [32, 160]
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2, 2, 0),  # [32, 80]
        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 80, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )
    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)